In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import scipy.stats as stats
from IPython.display import display
import pandas as pd
from utils import *
import gc

In [2]:
@timer_func
@multi_thread_wrapper
@multi_input_wrapper
@compute_CIR
def Algo_2nd_CIR(sigma,a,k,n,x0,N):
    return CIR_2nd(sigma,a,k,n,x0,N)

@timer_func
@multi_thread_wrapper
@multi_input_wrapper
@compute_CIR
def Algo_3rd_CIR(sigma,a,k,n,x0,N):
    return CIR_3rd(sigma,a,k,n,x0,N)

@timer_func
@multi_thread_wrapper
@multi_input_wrapper
@compute_CIR
def Exact_(sigma,a,k,n,x0):
    return Exact(sigma,a,k,n,x0)

@timer_func
@multi_thread_wrapper
@multi_input_wrapper
@compute_CIR
def Euler(sigma,a,k,n,x0,N,method):
    return{
            "Euler_DD":Euler_DD,
            "Euler_HM":Euler_HM,
            "Euler_L": Euler_L,
            "Euler_B":Euler_B}[method](sigma,a,k,n,x0,N)

In [3]:
def error(a,k,x0,sigma,N,t,n,confidence,method,submethod = None):
    alpha=1-confidence
    Z=stats.norm.ppf(1-alpha/2)

    exact_value = exact_expectation(x0,a,sigma,k,t)
    
    if method == Exact_:
        param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n}
    elif method == Euler:
        param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N,'method':submethod}
    elif method == Algo_2nd_CIR or method == Algo_3rd_CIR:
        param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N}
    else:
        print("No such method!")
        return None
    

    result=esperance_CIR(1)(method)([param_dict])
    mean= np.mean(result)
    var = np.var(result)
    del result

    interval_l = mean-Z*np.sqrt(var/N)
    interval_r = mean+Z*np.sqrt(var/N)

    appro_error = 1-mean/exact_value
    simul_error = np.sqrt(var/N)*Z/exact_value
    relative_error_l = abs(1-interval_l/exact_value)
    relative_error_r = abs(1-interval_r/exact_value)
    max_relative_error = max(relative_error_l,relative_error_r)
    print("The exact value of expectation is : {}".format(exact_value))
    print("The mean value of simulations is {}".format(mean))
    print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
    print("The maximal relative error of mean with {}% confidence is {}({}+-{})".format(confidence,max_relative_error,appro_error,simul_error))

# Modele CIR

In [4]:
a = 0.04
k = 0.1
x0 = 0.3
sigma = 2
t = 1

## Exact Simulation

In [5]:
N = 1000000
n = 1
confidence = 0.95
eps = 1e-3

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n}
result=esperance_CIR(1)(Exact_)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

appro_error = 1-mean/exact_value
simul_error = np.sqrt(var/N)*Z/exact_value
relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}({}+-{})".format(confidence,max_relative_error,appro_error,simul_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

Function 'Exact_' executed in 0.1790s
The exact value of expectation is : 0.8915304718347317
The absolute error tolerance is 0.001, the relative risk tolerance is 0.0011216666525621303
The mean value of simulations is 0.8915840208479568
The 0.95% confidence interval of mean2 is [0.8910628177026508,0.8921052239932629]
The maximal relative error of mean with 0.95% confidence is 0.0006446803297126547(-6.006414241221947e-05+-0.0005846161873002501)
maximal relative error of mean <= relative tolerance error? True


In [6]:
N = 100000000
n = 1
confidence = 0.95
eps = 1e-4

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n}
result=esperance_CIR(1)(Exact_)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

appro_error = 1-mean/exact_value
simul_error = np.sqrt(var/N)*Z/exact_value
relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}({}+-{})".format(confidence,max_relative_error,appro_error,simul_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

Function 'Exact_' executed in 18.6066s
The exact value of expectation is : 0.8915304718347317
The absolute error tolerance is 0.0001, the relative risk tolerance is 0.00011216666525621303
The mean value of simulations is 0.8915175796818454
The 0.95% confidence interval of mean2 is [0.8914654378392288,0.891569721524462]
The maximal relative error of mean with 0.95% confidence is 7.294646403843696e-05(1.4460697972284997e-05+-5.8485766066177595e-05)
maximal relative error of mean <= relative tolerance error? True


# Euler

In [7]:
N = 1000000
n = 1000
confidence = 0.95
eps = 1e-3

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N,'method':"Euler_DD"}
result=esperance_CIR(1)(Euler)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

appro_error = 1-mean/exact_value
simul_error = np.sqrt(var/N)*Z/exact_value
relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}({}+-{})".format(confidence,max_relative_error,appro_error,simul_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

Function 'Euler' executed in 96.1557s
The exact value of expectation is : 0.8915304718347317
The absolute error tolerance is 0.001, the relative risk tolerance is 0.0011216666525621303
The mean value of simulations is 0.8920416248842282
The 0.95% confidence interval of mean2 is [0.8915190770377504,0.8925641727307061]
The maximal relative error of mean with 0.95% confidence is 0.0011594678237381029(-0.0005733433299757174+-0.0005861244937623732)
maximal relative error of mean <= relative tolerance error? False


In [8]:
N = 1000000
n = 100
confidence = 0.95
eps = 1e-4

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N,'method':"Euler_DD"}
result=esperance_CIR(1)(Euler)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}".format(confidence,max_relative_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

Function 'Euler' executed in 12.1997s
The exact value of expectation is : 0.8915304718347317
The absolute error tolerance is 0.0001, the relative risk tolerance is 0.00011216666525621303
The mean value of simulations is 0.8983287723293057
The 0.95% confidence interval of mean2 is [0.8977931007950899,0.8988644438635216]
The maximal relative error of mean with 0.95% confidence is 0.008226271855517053
maximal relative error of mean <= relative tolerance error? False


# 2nd

In [9]:
N = 1000000
n = 16
confidence = 0.95
eps = 1e-3

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N}
result=esperance_CIR(1)(Algo_2nd_CIR)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}".format(confidence,max_relative_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

Function 'Algo_2nd_CIR' executed in 5.9127s
The exact value of expectation is : 0.8915304718347317
The absolute error tolerance is 0.001, the relative risk tolerance is 0.0011216666525621303
The mean value of simulations is 0.8911409735415591
The 0.95% confidence interval of mean2 is [0.8906217370163374,0.8916602100667808]
The maximal relative error of mean with 0.95% confidence is 0.0010192975418149564
maximal relative error of mean <= relative tolerance error? True


In [ ]:
N = 100000000
n = 33
confidence = 0.95
eps = 1e-4

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N}
result=esperance_CIR(1)(Algo_2nd_CIR)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}".format(confidence,max_relative_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

# 3rd

In [ ]:
N = 1000000
n = 16
confidence = 0.95
eps = 1e-3

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N}
result=esperance_CIR(1)(Algo_3rd_CIR)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

appro_error = 1-mean/exact_value
simul_error = np.sqrt(var/N)*Z/exact_value
relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}({}+-{})".format(confidence,max_relative_error,appro_error,simul_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))

In [ ]:
N = 100000000
n = 30
confidence = 0.95
eps = 1e-4

alpha=1-confidence
Z=stats.norm.ppf(1-alpha/2)

exact_value = exact_expectation(x0,a,sigma,k,t)
relative_tolerance = eps/exact_value

param_dict={'x0':np.array([x0]*N),'k':k,'a':a,'sigma':sigma,'n':n,'N':N}
result=esperance_CIR(1)(Algo_3rd_CIR)([param_dict])
mean= np.mean(result)
var = np.var(result)
del result

interval_l = mean-np.sqrt(var/N)*Z
interval_r = mean+np.sqrt(var/N)*Z

appro_error = 1-mean/exact_value
simul_error = np.sqrt(var/N)*Z/exact_value
relative_error_l = abs(1-interval_l/exact_value)
relative_error_r = abs(1-interval_r/exact_value)
max_relative_error = max(relative_error_l,relative_error_r)
print("The exact value of expectation is : {}".format(exact_value))
print("The absolute error tolerance is {}, the relative risk tolerance is {}".format(eps,relative_tolerance))
print("The mean value of simulations is {}".format(mean))
print("The {}% confidence interval of mean2 is [{},{}]".format(confidence,interval_l,interval_r))
print("The maximal relative error of mean with {}% confidence is {}({}+-{})".format(confidence,max_relative_error,appro_error,simul_error))
print("maximal relative error of mean <= relative tolerance error? {}".format(max_relative_error<=relative_tolerance))